In [1]:
import pandas as pd
import pymongo                                                  
from bson.objectid import ObjectId
from dotenv import load_dotenv
from pymongo import MongoClient
import os
import pprint

In [2]:
# Connect to your MongoDB database
load_dotenv()
uri = os.getenv('URI')
client = pymongo.MongoClient(uri)
db = client['nba_stats']
hist_player_stats = db['hist_player_logs']

# creating pandas df for players stats
hist_player_stats = db['hist_player_logs']
player_cursor = hist_player_stats.find()
hist_players_df = pd.DataFrame(list(player_cursor))
# all_players_df = all_players_df.drop(columns='_id')
print(len(hist_players_df))


163237


In [11]:
# Assuming player_stats is your MongoDB collection object
pipeline = [
    {"$group": {"_id": {"unique_id":"$unique_id", "season_id":"$season_id","game_date": "$game_date", "player_id": "$player_id", "min": "$min", "pts": "$pts", "reb": "$reb", "ast": "$ast", 
                        "stl": "$stl", "blk": "$blk", "player_game_no": "$player_game_no"}, "unique_ids": {"$addToSet": "$_id"}, "count": {"$sum": 1}}},
    {"$match": {"count": {"$gt": 1}}}
]

# Find duplicate documents
duplicate_docs = list(hist_player_stats.aggregate(pipeline)) #use the mongo db/collection variable not the df
print(len(duplicate_docs))
pprint.pprint(duplicate_docs)

# sort duplicate documents
# duplicate_docs = sorted(duplicate_docs, key=lambda x: x['_id']['player_id'])
# duplicate_docs = sorted(duplicate_docs, key=lambda x: x['_id']['game_date'])




0
[]


In [ ]:
# Delete duplicate documents except for the first occurrence
for index, doc in enumerate(duplicate_docs):
    unique_ids = doc["unique_ids"]
    del unique_ids[0]  # Keep the first occurrence, delete the rest
    hist_players_df.delete_many({"_id": {"$in": unique_ids}})
    # print(index)